In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
data_15 = pd.read_csv('/kaggle/input/world-happiness/2015.csv')
data_16 = pd.read_csv('/kaggle/input/world-happiness/2016.csv')
data_17 = pd.read_csv('/kaggle/input/world-happiness/2015.csv')

In [ ]:
data_15.head(10)

In [ ]:
data_15.info()

In [ ]:
data_15.describe()

In [ ]:
data_16.describe()

In [ ]:
data_17.describe()

The standard error seems not to be big so drop it
Should do another one w/ standard deviation if there is more accuracy

In [ ]:
data_15.drop(['Standard Error'],axis=1,inplace=True)
data_17.drop(['Standard Error'],axis=1,inplace=True)


In [ ]:
data_16.drop(['Lower Confidence Interval','Upper Confidence Interval'],axis=1,inplace=True)

In [ ]:
data_15.describe()


In [ ]:
data_16.describe()

In [ ]:
data_17.describe()

There seems to be not much change in the mean happiness of the world over time

In [ ]:
data = pd.concat([data_15,data_16,data_17])
data.rename(columns={'Economy (GDP per Capita)':'GDP/Capita'},inplace=True)

In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
data['Country'].nunique()

In [ ]:
data.groupby('Region').mean().sort_values(by='Happiness Score',ascending=False)

In [ ]:
data.groupby('Region').mean().sort_values(by='Happiness Rank')

It is not surprising that Western countries are on the top rank of hapiness. But is this because there are more data of the one country than another. Like the # of countries in Asia are bigger than ones of Western region

In [ ]:
plt.figure(figsize=(25,5))
sns.countplot('Region',data=data )

Let's compare the number of countries in each region and that region's happiness rank

In [ ]:
data[['Region','Happiness Rank']].groupby('Region').aggregate(['count', np.mean]).sort_values(by=[('Happiness Rank','mean')])

In [ ]:
data[['Region','Happiness Rank']].groupby('Region').aggregate(['count', np.mean]).corr()

There seems to be a slight correlation between the number of countries in a region and the average ranking of that region. But this correlation is not significant enough

I am an Asian so I would be interested in looking at the Countries in Asia

In [ ]:
data[data['Country']=='Vietnam']

Just a little bit curious about the ranking of my country. Look like Vietnam change its happiness rank over the year

In [ ]:
sns.jointplot('Generosity','Happiness Rank',data=data, kind = 'reg')

Hmm. Seems like there is no correlation between the happiness ranking and generosity

Let's visualize how many countries in each region

And visualize the distribution of the contribution of GDP/Capita to Happiness Scores

In [ ]:
sns.set_style('whitegrid')
sns.distplot(data['GDP/Capita'])

In [ ]:
data.set_index(['Region','Country'])

In [ ]:
data[['Region','GDP/Capita','Happiness Rank']].groupby(by='Region',as_index=False).mean().sort_values(by='GDP/Capita',ascending=False)

Find out correlation between GDP/Capita and Happiness Rank

In [ ]:
data[['GDP/Capita','Happiness Rank']].corr()

There seems to be a strong negative correlation between GDP/Capita and Happiness Rank, which is not surprising. Let's visualize this relationship

In [ ]:
sns.jointplot('GDP/Capita','Happiness Rank',data=data, kind = 'reg')

Let's find the correlation between each feature compared to the happiness score using heatmap

In [ ]:
sns.heatmap(data.corr(),annot=True)

Now let's look at the correlation in each region. Let's start with Australia and New Zealand

In [ ]:
sns.heatmap(data[data['Region']=='Australia and New Zealand'].corr(),annot=True)

In Australia and New Zealand region, Generosity and Health are the most important factors to happiness. It is interesting how small GDP contributes to happiness. 

Now let's look at North America

In [ ]:
sns.heatmap(data[data['Region']=='North America'].corr(),annot=True)

Interestingly, Trust and Freedom are the 2 most important factors to happiness in North America. Family is the least significant factor here. This is not surprising considering North America value individuality

How about Western Europe?

In [ ]:
sns.heatmap(data[data['Region']=='Western Europe'].corr(),annot=True)

Trust, freedom are also the 2 most important factors. There also seem to be strong correlation with other factors such as generosity, family, GDP

Southeastern Asia would also be interesting to look at

In [ ]:
sns.heatmap(data[data['Region']=='Southeastern Asia'].corr(),annot=True)

Since many countries in Southeastern Asia are develop countries, it is not surprising that GDP is the most important factor. Other important factors are GDP, family, and health. Considering how much people here care about their family connection, it is understandable to see the significant of family to happiness here. It is also worthy to notice how insignificant freedom in this region compared to Western countries

Would be interesting to look at Middle East and Northern Africa region

In [ ]:
sns.heatmap(data[data['Region']=='Middle East and Northern Africa'].corr(),annot=True)

Let's be excited to look at Latin America and Caribbean

In [ ]:
sns.heatmap(data[data['Region']=='Latin America and Caribbean'].corr(),annot=True)

On contrast to Western region such as North America or Western Europe, Trust, and Freedom the most insignificant factor to happiness. So fascinating how dinamic the change in the factors that contribute to happiness are in different countries

GDP is the most significant factor. Except generosity, many other factor also seem to contribute significantly to happiness of the population in this region. It is worthy to notice how insignificantly generosity is.

Let's look at the correlation between hapiness score and other features since that is what we care about.
* It is not surprising that there is a strong correlation between hapiness rank and hapiness score
* Ranking of features that is correlated to happiness score from highest to lowest:
 1. GDP/Capita
 2. Health
 3. Family
 4. Freedom
 5. Dystopia Residual
 6. Trust
 7. Generosity
The 3 most significant factors are money, health, and family. That sounds right. 

Let's compare the data from 3 different years and evaluate the change in hapiness ranking of a country over time

In [ ]:
df = data_15
df_1=pd.merge(df,data_16,on='Country')
df_1 = df_1[['Country','Happiness Rank_x','Happiness Score_x','Happiness Rank_y', 'Happiness Score_y']]

df_1

In [ ]:
df_1['Difference in rank'] = df_1['Happiness Rank_x'].sub(df_1['Happiness Rank_y'])
df_1['Difference in score'] = df_1['Happiness Score_x'].sub(df_1['Happiness Score_y'])
df_1

In [ ]:
sns.distplot(df_1['Difference in rank'])

In [ ]:
sns.distplot(df_1['Difference in score'])

In [ ]:
df_1['Difference in rank'].describe()

In [ ]:
df_1['Difference in score'].describe()

The majority of difference in rank and score of data from 2015 and 2016 is at 0. This indicates that the rank and the score tends to be consistent from 2015 to 2016. 
Let's find out which countries have a signicant decrease in its rank

In [ ]:
df_1[df_1['Difference in rank']>10][['Country','Happiness Rank_x','Happiness Rank_y','Difference in rank']].sort_values(by='Difference in rank',ascending=False)

13 countries with the increase in rank ranging from 12-30 in one year.

Now let's observe the countries with the significant increase in rank from 2015 to 2016

In [ ]:
df_1[df_1['Difference in rank']<-10][['Country','Happiness Rank_x','Happiness Rank_y','Difference in rank']].sort_values(by='Difference in rank')